In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import warnings
warnings.filterwarnings('ignore')

In [2]:
network_input = np.load('./../NumpyDataset/Input-Tensor.npy')
network_output = np.load('./../NumpyDataset/Output-Tensor.npy')
print(network_input.shape, network_output.shape)

(57077, 100, 1) (57077, 358)


In [6]:
g = tf.Graph()
time_steps=100
with g.as_default():
    x = tf.placeholder(dtype=tf.float32, 
                       shape=(None, 100, 1))
    y_true = tf.placeholder(dtype=tf.float32, 
                       shape=(None, 358))
    y_true_cls = tf.argmax(y_true, dimension=1)
    inpt=tf.unstack(x ,time_steps,1)
    num_units = [256, 512, 256]
    rnn_layers = []
    for size in num_units:
        
        rnn_layers.append(tf.nn.rnn_cell.LSTMCell(size, 
                                                  forget_bias=1)) 
        
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
    
    outputs, states = tf.nn.static_rnn(cell=multi_rnn_cell, 
                                       inputs=inpt,
                                       dtype=tf.float32)
    
    output_dense1 = tf.layers.dense(outputs[-1], 
                                    256, 
                                    activation=None)
    
    logits = tf.layers.dense(output_dense1, 
                                    358, 
                                    activation=None)
    
    y_pred = tf.nn.softmax(logits=logits)
    
    y_pred_cls = tf.argmax(y_pred, 
                           dimension=1)
    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, 
                                                            logits=logits)
    loss = tf.reduce_mean(cross_entropy)
    
    optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-4).minimize(loss)
    
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    
logits

<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 358) dtype=float32>

In [8]:
tf.summary.FileWriter("logs", g).close()